# OpenSTL fUS NPZ Integration (Step-by-Step)

Run each section one cell at a time. This notebook keeps the workflow fully interactive.

## 1) Environment Setup

In [ ]:
import subprocess
import sys
from pathlib import Path


def find_repo_root(start: Path) -> Path | None:
    """Walk upward from `start` and return the first fUSPredict-like root."""
    for p in [start, *start.parents]:
        if (p / 'code').exists() and (p / 'derivatives').exists():
            return p
    return None


cwd = Path.cwd().resolve()
repo_root = find_repo_root(cwd)

# Fallback: set this explicitly if your Jupyter working directory is elsewhere.
if repo_root is None:
    manual_repo_root = Path(r'C:\Users\ESPCI\Documents\GitHub\fUSPredict')
    if (manual_repo_root / 'code').exists() and (manual_repo_root / 'derivatives').exists():
        repo_root = manual_repo_root.resolve()

if repo_root is None:
    raise RuntimeError(
        'Could not locate fUSPredict root from current working directory. '        f'cwd={cwd}. Please set manual_repo_root in this cell.'
    )

for p in (str(repo_root), str(repo_root / 'code')):
    if p not in sys.path:
        sys.path.insert(0, p)

print('cwd       =', cwd)
print('repo_root =', repo_root)



## 2) Choose Manifest

In [ ]:
# manifest_path = repo_root / 'derivatives' / 'preprocessing' / 'splits_multi.json'
manifest_path = repo_root / 'derivatives' / 'preprocessing' / 'splits_single_secundo.json'
manifest_path

## 3) Validate Manifest Paths (Windows-safe JSON roundtrip)

In [ ]:
from utils.manifest_utils import assert_manifest_paths_exist

n_checked = assert_manifest_paths_exist(manifest_path)
print(f'Validated {n_checked} manifest path entries')

## 4) Local Dataset Smoke Test (x/y shapes + deterministic eval)

In [ ]:
import torch

sys.path.insert(0, str(repo_root / 'code' / '3_modeling' / 'openstl_patch'))
from openstl.datasets.fus_npz_dataset import FUSNPZForecastDataset

pre_seq_length = 20
aft_seq_length = 20

ds_train = FUSNPZForecastDataset(
    manifest_path=str(manifest_path),
    split='train',
    pre_seq_length=pre_seq_length,
    aft_seq_length=aft_seq_length,
    random_window=True,
    seed=42,
)
ds_test = FUSNPZForecastDataset(
    manifest_path=str(manifest_path),
    split='test',
    pre_seq_length=pre_seq_length,
    aft_seq_length=aft_seq_length,
    random_window=False,
    seed=42,
)

x_tr, y_tr = ds_train[0]
x_te_1, y_te_1 = ds_test[0]
x_te_2, y_te_2 = ds_test[0]

assert x_tr.dtype == torch.float32 and y_tr.dtype == torch.float32
assert torch.equal(x_te_1, x_te_2) and torch.equal(y_te_1, y_te_2)

print('train len:', len(ds_train), '| test len:', len(ds_test))
print('x shape:', tuple(x_tr.shape), '| y shape:', tuple(y_tr.shape))
print('deterministic eval: passed')

## 5) Set OpenSTL Repo Path

In [ ]:
def is_openstl_root(p: Path) -> bool:
    return (
        (p / 'openstl').is_dir()
        and (p / 'configs').is_dir()
        and (p / 'tools' / 'train.py').is_file()
    )


candidates = [
    Path(r'C:\Users\ESPCI\OpenSTL'),
    Path(r'C:\path\to\OpenSTL'),
]

openstl_root = None
for cand in candidates:
    if is_openstl_root(cand):
        openstl_root = cand.resolve()
        break

if openstl_root is None:
    raise RuntimeError(
        'Could not find OpenSTL root automatically. '        'Set `openstl_root` to your OpenSTL repo folder in this cell.'
    )

print('openstl_root =', openstl_root)



## 6) Apply OpenSTL Patch Files

In [ ]:
patch_script = repo_root / 'code' / '3_modeling' / 'openstl_patch' / 'apply_openstl_fus_patch.py'
cmd = [sys.executable, str(patch_script), '--openstl-root', str(openstl_root)]
print('Running:', ' '.join(cmd))

result = subprocess.run(cmd, text=True, capture_output=True)
print('returncode:', result.returncode)
if result.stdout:
    print('--- stdout ---')
    print(result.stdout)
if result.stderr:
    print('--- stderr ---')
    print(result.stderr)

if result.returncode != 0:
    raise RuntimeError('Patch application failed. See stdout/stderr above.')



## 7) Inspect Generated OpenSTL Config

In [ ]:
config_path = openstl_root / 'configs' / 'fus' / 'SimVP.py'
print(config_path)
print(config_path.read_text(encoding='utf-8'))

## 8) Train / Evaluate Commands (run in OpenSTL terminal)

Train:
```powershell
python tools\train.py --config_file configs\fus\SimVP.py --dataname fus_npz --method SimVP
```

Evaluate:
```powershell
python tools\train.py --config_file configs\fus\SimVP.py --dataname fus_npz --method SimVP --test --ckpt_path C:\path\to\ckpt.ckpt
```